# Python Plots for LGalaxies

## Import Libraries and Read Catalogs

<p>Use functions read_snap or read_tree to read catalogs. These are both defined in procedures.py. In case of read_snap, SnapshotList will be returned containing the list of snapshots read (usefull to later select galaxies in a given redshift).<p>

In [1]:
import numpy as np
%matplotlib inline

import pandas as pd

%pylab inline
#import seaborn as sns
#sns.set_style('darkgrid')

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from astropy.table import Table
from importlib import reload
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LogNorm
from decimal import *
from matplotlib import animation
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import zoom 

import procedures
reload (procedures)
from procedures import *
import plots_input
reload (plots_input)
from plots_input import *

FirstFile = 0
LastFile =  19

Volume_MR = (BoxSize_MR**3.0) * (LastFile - FirstFile + 1) / MaxTreeFiles 
Volume_MRII = (BoxSize_MRII**3.0) * (LastFile - FirstFile + 1) / MaxTreeFiles 

if CatalogType=='snap':
    from LGalaxies_Henriques2015a_struct import LGalaxiesStruct
    from LGalaxies_Henriques2015a_struct import PropertiesToRead
    (G_MR, SnapshotList) = read_snap(DirName_MR,FirstFile,LastFile,
                     PropertiesToRead,LGalaxiesStruct,RedshiftsToRead,RedshiftList)
    
if CatalogType=='tree':    
    from LGalaxies_tree_Henriques2015a_struct import LGalaxiesStruct
    from LGalaxies_tree_Henriques2015a_struct import PropertiesToRead_tree    
    (G_MR) = read_tree(DirName_MR,FirstFile,LastFile,
                     PropertiesToRead_tree,LGalaxiesStruct,RedshiftsToRead,RedshiftList)    
     
    SnapshotList = np.zeros(len(RedshiftList),dtype=np.int32)
    for ii in range(0,len(RedshiftList)):                  
        G0=G_MR[ np.rint(G_MR['Redshift']*100.) == RedshiftList[ii]*100. ]             
        SnapshotList[ii]=G0['SnapNum'][0]
#endif      

print('reading done\n')
#print (np.log10(gal['StellarMass'][1:5]*1.e10))
#help(gal)



Populating the interactive namespace from numpy and matplotlib


Reading Headers

TotNgals= [37277179]


File  0  nGals =  [2332354]
File  1  nGals =  [1432596]
File  2  nGals =  [1419994]
File  3  nGals =  [1197118]
File  4  nGals =  [1724741]
File  5  nGals =  [1872305]
File  6  nGals =  [2455764]
File  7  nGals =  [2079173]
File  8  nGals =  [1483950]
File  9  nGals =  [1571856]
File  10  nGals =  [2320518]
File  11  nGals =  [1869883]
File  12  nGals =  [2231795]
File  13  nGals =  [2097441]
File  14  nGals =  [1833687]
File  15  nGals =  [1742179]
File  16  nGals =  [1650730]
File  17  nGals =  [2860344]
File  18  nGals =  [1932021]
File  19  nGals =  [1168730]
reading done



## Plots

In [56]:
plt.rcParams.update({'font.size': 18, 'xtick.labelsize': 18, 'ytick.labelsize': 18, 'axes.linewidth': 2, 
                     'xtick.major.size': 6, 'xtick.major.width': 1.5, 
                     'ytick.major.size': 6, 'ytick.major.width': 1.5, 
                     'xtick.minor.size': 3, 'xtick.minor.width': 1.,                   
                     'ytick.minor.size': 3, 'ytick.minor.width': 1.})

In [ ]:
with PdfPages('./fig/plots.pdf') as pdf:
    import plots
    reload (plots)
    
    if opt_stellar_mass_function==1:
        from plots import stellar_mass_function
        stellar_mass_function(G_MR, SnapshotList, Volume_MR, pdf)
        
    if opt_bluck_red_fractions==1:
        from plots import bluck_red_fractions
        bluck_red_fractions(G_MR, SnapshotList, pdf)
             
    if opt_test_plots==1:
        from plots import test_plots     
        test_plots(G_MR, SnapshotList, pdf)
        
#end with PdfPages('./fig/plots.pdf') as pdf:

opt_anime_mass_gr=1
if opt_anime_mass_gr==1:   
    
    from scipy.ndimage import zoom 
    from procedures import *
    
    xlim=[9.7,11.5]
    ylim=[0.1, 1.0]
    bin=[0.1,0.05]
    Nbins=[int((xlim[1]-xlim[0])/bin[0]),int((ylim[1]-ylim[0])/bin[1])]
    
    fig = plt.figure(figsize=(13,9))    
    subplot=plt.subplot()
    subplot.set_ylim(ylim), subplot.set_xlim(xlim)        
    #ax = plt.axes(xlim=xlim, ylim=ylim)      
    xlab='$log_{10}(M_*[M_{\odot}])$'           
    ylab='$g-r$'               
    subplot.set_xlabel(xlab, fontsize=16), subplot.set_ylabel(ylab, fontsize=16)  
    yy= np.linspace(0.,1.,1000.)    
    # animation function
    
    def animate(nframe): 
        print('doing frame:',nframe)  
        #clear previous image
        clf()
        subplot=plt.subplot()
        subplot.set_ylim(ylim), subplot.set_xlim(xlim)         
        subplot.set_xlabel(xlab, fontsize=16), subplot.set_ylabel(ylab, fontsize=16)               
        x=[xlim[0],xlim[1]]    
        #fill the entire plot area to delete previous frame, plot axis
        subplot.fill_between(x, [ylim[1],ylim[1]], [ylim[0],ylim[0]], facecolor='lavender')                
        for jj in range (1,10):
            value=9.5++jj/4.
            shift=0.003
            subplot.fill_between([value-shift,value+shift], [ylim[1],ylim[1]], [ylim[0],ylim[0]], facecolor='white',edgecolor="w")
            value=jj/10.
            shift=0.002            
            subplot.fill_between(x, [value+shift,value+shift], [value-shift,value-shift], facecolor='white',edgecolor="w")
                  
        #label        
        G0=G_MR[G_MR['SnapNum']==nframe+20] 
        props = dict(boxstyle='round', facecolor='ghostwhite')
        subplot.text(10.83, 0.165, 'z='+'%0.2f\n' % G0['Redshift'][0] 
                     + 'Age='+'%0.2f' % redshift_to_time(G0['Redshift'][0])+' Gyr', 
                     fontsize= 24, bbox=props)
            
        #ignore first 20 snaps which have no galaxies 
        G0_MR=G_MR[(G_MR['StellarMass']>0.1) & (G_MR['SnapNum']==nframe+20)]  
        Ngals=len(G0_MR)
        StellarMass=(np.log10(G0_MR['StellarMass']*1.e10/Hubble_h))     
        color_gr=G0_MR['MagDust'][:,16]-G0_MR['MagDust'][:,17]                   
        H, xedges, yedges = np.histogram2d(StellarMass, color_gr, bins=Nbins)            
        extent = [xedges[0], xedges[-1],yedges[0], yedges[-1]]       
        subplots_adjust(bottom=0.15, left=0.15)        
        mylevels = np.linspace(1., Nbins[0], Nbins[0])*Ngals/(Nbins[0]**2/0.7)        
        H = zoom(H, 20)    
        cont=contourf(H.transpose()[::], origin='lower', cmap='rainbow', levels=mylevels, extent=extent)        
        plt.colorbar()
        return cont  
    #end animate
    
    anim = animation.FuncAnimation(fig, animate, frames=39, blit=True)
    #plt.colorbar()
    anim.save('mass_gr.m4v', fps=1)
    
print('finished')
  

doing frame: 0
doing frame:

/Users/BrunoHenriques/anaconda/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
/Users/BrunoHenriques/anaconda/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


 0
doing frame: 1
doing frame: 2
doing frame: 3
doing frame: 4
doing frame: 5
doing frame: 6
doing frame: